In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import matplotlib.pyplot as plt
import sys,os
from os.path import join
sys.path.insert(0, '/home/monte.flora/python_packages/scikit-explain/')
import skexplain 

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.common.util import subsampler, normalize_importance, compute_sage
from sklearn.datasets import make_classification
import shap
import matplotlib.pyplot as plt
import xarray as xr 

In [2]:
def create_model(N=5000):
    np.random.seed(42)  # For reproducibility

    # Generate random values for two features
    feature1 = np.random.rand(N)
    feature2 = np.random.rand(N)

    # Generate the interaction term by multiplying the two features
    interaction = feature1 * feature2

    # Generate the target variable (dependent variable) as a linear combination of features and interaction
    y = 0 * feature1 + 0 * feature2 + 10 * interaction 

    # Create a DataFrame to hold the dataset
    X = pd.DataFrame({'Feature 1': feature1, 'Feature 2': feature2})

    est = RandomForestRegressor(max_features=None).fit(X,y)
    #est = LinearRegression().fit(X,y)
    
    explainer = skexplain.ExplainToolkit(('LR', est), X=X, y=y)
    
    return explainer, X

explainer, X = create_model()

In [7]:
# Calculate the H-statistics
def compute_H2jk(PD_jk, PD_j, PD_k):

    n_boot = len(PD_j)
    h_stat = np.zeros((n_boot))
    
    for i in range(n_boot):
        numerator = np.sum(np.square(PD_jk[i] - PD_j[i] - PD_k[i]))
        denominator = np.sum(np.square(PD_jk[i]))

        h_stat[i] = np.sqrt(numerator / denominator)

    return h_stat

def h_squared_shap(explain_vals, X, features, n_bins=10):
    
    feature_vals_1 = X[features[0]].values
    feature_vals_2 = X[features[1]].values
    
    # Define the bins
    bins_1 = np.unique(
                np.percentile(feature_vals_1, np.linspace(0.0, 100.0, n_bins + 1), method="lower"
                ))
    bins_2 = np.unique(
                np.percentile(feature_vals_2, np.linspace(0.0, 100.0, n_bins + 1), method="lower"
                ))

    # Get the bin number for each point
    inds_1 = np.clip(np.digitize(feature_vals_1, bins_1, right=True) - 1, 0, None)
    inds_2 = np.clip(np.digitize(feature_vals_2, bins_2, right=True) - 1, 0, None)

    # Initialize arrays to hold the average explainability values and interaction values
    mean_explain_vals = np.full((n_bins, n_bins), np.nan)
    sum_explain_vals = np.full((n_bins, n_bins), np.nan)
    interact_vals = np.full((n_bins, n_bins), np.nan)

    # Compute the average explainability values and interaction values in each bin
    for i in range(n_bins):
        for j in range(n_bins):
            vals_in_bin = explain_vals[(inds_1 == i) & (inds_2 == j)]
            if vals_in_bin.size > 0:
                # Approx first order effect
                mean_explain_vals[i, j] = np.nanmean(vals_in_bin)
                # Full effect.
                sum_explain_vals[i, j] = np.nansum(vals_in_bin)
                # Higher order effect.
                interact_vals[i, j] = np.nanmean(np.abs(vals_in_bin - mean_explain_vals[i, j]))

    H_squared = np.nansum(interact_vals**2) / np.nansum(sum_explain_vals**2) 
                
    return np.sqrt(H_squared)

In [4]:
shap = explainer.local_attributions(method='shap')

Exact explainer: 5001it [02:12, 36.21it/s]                                                                                                                                                         


NameError: name 'n_bins' is not defined

In [9]:
features = ['Feature 1', 'Feature 2']
h_squared_shap(shap.values, X, features)

TypeError: 'method' object is not subscriptable